In [1]:
from transformers import MusicgenForConditionalGeneration
from IPython.display import Audio
from transformers import AutoProcessor

In [2]:
!export CUDA_LAUNCH_BLOCKING=1

In [3]:
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")
sampling_rate = model.config.audio_encoder.sampling_rate

/home/lleisner/.local/lib/python3.11/site-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_po

In [5]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device);

In [6]:

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")

inputs = processor(
    text=["slow jazz ballad with soft piano and ambient background"],
    padding=True,
    return_tensors="pt",
)

audio_values = model.generate(**inputs.to(device), do_sample=True, guidance_scale=9, temperature=1.5, max_new_tokens=128)

Audio(audio_values[0].cpu().numpy(), rate=sampling_rate)

`torch.nn.functional.scaled_dot_product_attention` does not support having an empty attention mask. Falling back to the manual attention implementation. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.Note that this probably happens because `guidance_scale>1` or because you used `get_unconditional_inputs`. See https://github.com/huggingface/transformers/issues/31189 for more information.


In [7]:
# Step 1: Generate 5s from text
text_prompt = "a medieval battlefield tune with bagpipes and distant war drums echoing through the hills"
short_inputs = processor(text=[text_prompt], return_tensors="pt").to(device)

short_audio = model.generate(**short_inputs, do_sample=True, max_new_tokens=256)
short_np = short_audio[0].cpu().numpy()
short_np_mono = short_np.squeeze()

In [8]:
# Step 2: Use the short audio as audio prompt for continuation
inputs = processor(
    audio=short_np_mono,
    sampling_rate=sampling_rate,
    text=[text_prompt],
    return_tensors="pt",
    padding=True
).to(device)

extended_audio = model.generate(**inputs, do_sample=True, max_new_tokens=256)


In [9]:
Audio(extended_audio[0].cpu().numpy(), rate=sampling_rate)

In [ ]:
def generate(prompt_seq, max_new_tokens=256):
    initial_inputs = processor(text=[prompt_seq[0]], return_tensors="pt")
    initial_inputs = {k: v.to(device) for k, v in initial_inputs.items()}

    audio = model.generate(**initial_inputs, do_sample=True, max_new_tokens=max_new_tokens)
    
    for prompt in prompt_seq[1:]:
        audio_np_mono = audio[0].cpu().numpy().squeeze()
        inputs = processor(
            audio=audio_np_mono,
            sampling_rate=sampling_rate,
            text=[prompt],
            return_tensors="pt",
            padding=True
        ).to(device)

        audio = model.generate(**inputs, do_sample=True, max_new_tokens=max_new_tokens)
    
    return audio[0].cpu().numpy()
        

In [14]:
prompt_sequence = [
    "solo classical piano playing a romantic Chopin-style nocturne",
    "chamber string quartet with lush harmonies and expressive phrasing",
    "jazz trio with upright bass, brushed drums, and improvising piano",
    "bossa nova with nylon guitar, soft percussion, and laid-back groove",
    "70s soul with electric piano, funky bassline, and smooth vocals",
    "80s synth funk with slap bass, gated drums, and digital keys",
    "early house music with 909 drums, looping piano chords, and vocal chops",
    "modern EDM track with sidechained synths and rising drops",
    "trap-influenced pop with tuned vocals, 808s, and minimal melodies",
    "experimental hyperpop with glitched vocals, breakbeats, and bright synth overload"
]



In [15]:
Audio(generate(prompt_seq=prompt_sequence, max_new_tokens=128), rate=sampling_rate)